In [1]:
import requests
from bs4 import BeautifulSoup
import re
import io
import PyPDF2
import pandas as pd
import datetime
import time

In [ ]:
cur_year = datetime.datetime.now().year
cur_month = datetime.datetime.now().month
error_url = 'https://www.ftserussell.com/products/indices/home/errorview?aspxerrorpath=/Analytics/FactSheets/Home/DownloadSingleIssueByDate'
dates = []
report_links = []
marketCaps = []
for y in range(2013,cur_year+1):
    year = str(y)
    for m in range(1,13):
        if y==cur_year and m>cur_month:
            break
        if m < 10:
            month = '0' + str(m)
        else:
            month = str(m)
        for d in range(28,32):
            day = str(d)
            date = year + month + day
            url = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=GXUSS&IssueDate="+ date + "&IsManual=false.pdf"
            response = requests.get(url)
            if response.url != error_url:
                dates.append(date)
                pdf = requests.get(url)
                open_pdf = io.BytesIO(pdf.content)
                read_pdf = PyPDF2.PdfFileReader(open_pdf)
                pdf_page2 = read_pdf.getPage(1)
                page_text = pdf_page2.extractText()
                cleaned_page_text = page_text.replace('\n',' ').replace('  ',' ').replace('  ',' ')
                mc = re.findall('Totals \d+ \d+,\d+,\d+', cleaned_page_text, re.IGNORECASE)
                if mc == []:
                    mc = 'n/a'
                    marketCaps.append(mc)
                    continue
                else:
                    mc = mc[0].replace(',','').replace('\n','').replace('Totals','')
                    mc = [int(s) for s in mc.split() if s.isdigit()][1] / 1000
                    marketCaps.append(mc)
                break
vxusDF = pd.DataFrame({'date':pd.to_datetime(dates, infer_datetime_format=True),'Market Cap':marketCaps})
vxusDF.to_csv('Data/vxus_MC_data.csv',index=False)